In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# ============================================
#  oad from .env file (Recommended)
# ============================================
# Try to load from .env file in directory
env_path = Path("./.env")
if env_path.exists():
    load_dotenv(env_path)
    print("📄 Loaded configuration from .env file")
else:
    print("⚠️  No .env file found - using environment variables or hardcoded keys")

# ============================================
# Verify API keys are set
# ============================================
print("\n🔑 API Key Status:")
if os.getenv("HOLISTIC_AI_TEAM_ID") and os.getenv("HOLISTIC_AI_API_TOKEN"):
    print("  ✅ Holistic AI Bedrock credentials loaded (will use Bedrock)")
elif os.getenv("OPENAI_API_KEY"):
    print("  ⚠️  OpenAI API key loaded (Bedrock credentials not set)")
    print("     💡 Tip: Set HOLISTIC_AI_TEAM_ID and HOLISTIC_AI_API_TOKEN to use Bedrock (recommended)")
else:
    print("  ⚠️  No API keys found")
    print("     Set Holistic AI Bedrock credentials (recommended) or OpenAI key")

if os.getenv("VALYU_API_KEY"):
    key_preview = os.getenv("VALYU_API_KEY")[:10] + "..."
    print(f"  ✅ Valyu API key loaded: {key_preview}")
else:
    print("  ⚠️  Valyu API key not found - search tool will not work")

print("\n📁 Working directory:", Path.cwd())

# ============================================
# Import Holistic AI Bedrock helper function
# ============================================
# Import from core module (recommended)
import sys

try:
    sys.path.insert(0, "./core")
    from react_agent.holistic_ai_bedrock import HolisticAIBedrockChat, get_chat_model

    print("\n✅ Holistic AI Bedrock helper function loaded")
except ImportError:
    print("\n⚠️  Could not import from core - will use OpenAI only")
    print("   Make sure core/react_agent/holistic_ai_bedrock.py exists")

# Import official packages
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

print("\n✅ All imports successful!")

📄 Loaded configuration from .env file

🔑 API Key Status:
  ✅ Holistic AI Bedrock credentials loaded (will use Bedrock)
  ✅ Valyu API key loaded: Pcet7Qftke...

📁 Working directory: /Users/zachmothner/UCL/BiasSphere/models

✅ Holistic AI Bedrock helper function loaded

✅ All imports successful!


In [6]:
# import time
# from langgraph.prebuilt import create_react_agent
# from langchain_core.messages import HumanMessage

# # Example 2: ReAct Agent (no tools yet)
# print("="*70)
# print("EXAMPLE 2: ReAct Agent (No Tools)")
# print("="*70)

# # Create LLM using helper function
# llm = get_chat_model("claude-3-5-sonnet")

# # Create ReAct agent with NO tools (for now)
# agent = create_react_agent(llm, tools=[])

# # Use the same question from Example 1
# question = "What is quantum computing?"
# print(f"\n❓ Same Question: {question}")

# start_time = time.time()
# result = agent.invoke({"messages": [HumanMessage(content=question)]})
# elapsed = time.time() - start_time

# print(f"\n💬 Response: {result['messages'][-1].content}")
# print(f"\n⏱️  Time: {elapsed:.2f}s")
# print(f"📊 Messages in conversation: {len(result['messages'])}")
# print("\n✅ Agent can maintain context, use tools (when provided), and handle multi-turn!")

# Import official Valyu tool from langchain-valyu package
from langchain_valyu import ValyuSearchTool

# Create search tool with configuration
search_tool = ValyuSearchTool(
    valyu_api_key=os.getenv("VALYU_API_KEY"),
    # Optional: configure search parameters (can also be set per-call)
    # search_type="all",  # Search both proprietary and web sources
    # max_num_results=5,   # Limit results
    # relevance_threshold=0.5,  # Minimum relevance score
    # max_price=20.0  # Maximum cost in dollars
)

# Test it directly
print("🔍 Testing Valyu Search Tool Directly")
print("=" * 70)
test_query = "latest developments in quantum computing"
print(f"Query: {test_query}\n")

# Call the search tool
search_results = search_tool._run(query=test_query, search_type="all", max_num_results=5)

# Display results (truncated for readability)
result_str = str(search_results)
print("📄 Search Results (first 500 chars):")
print("-" * 70)
print(result_str[:500] + "..." if len(result_str) > 500 else result_str)
print("-" * 70)
print(f"\n📊 Total data returned: {len(result_str)} characters")
print("✅ This data will be passed to the agent to answer questions!")

🔍 Testing Valyu Search Tool Directly
Query: latest developments in quantum computing

📄 Search Results (first 500 chars):
----------------------------------------------------------------------
{
  "success": true,
  "error": "",
  "tx_id": "tx_70d2997c-b9f8-4bc9-bdcc-22624f415a14",
  "query": "latest developments in quantum computing",
  "results": [
    {
      "title": "News - Quantum Computing Report",
      "url": "https://quantumcomputingreport.com/news/?utm_source=valyu.ai&utm_medium=referral",
      "content": "# News\n\nRecent news items published within the last 6 months on quantum computing developments are listed below. Click on the hyperlinked item to go to the press relea...
----------------------------------------------------------------------

📊 Total data returned: 94655 characters
✅ This data will be passed to the agent to answer questions!


In [7]:
# Test question about recent events (requires current information)
recent_question = "What are the latest breakthroughs in quantum computing in 2025?"

# print("="*70)
# print("TEST 1: Agent WITHOUT Search")
# print("="*70)
# print(f"❓ Question: {recent_question}\n")

# start_time = time.time()
# result_no_search = agent.invoke({
#     "messages": [HumanMessage(content=recent_question)]
# })
# elapsed_no_search = time.time() - start_time

# print("💬 Response (NO search):")
# print("-"*70)
# response_text = result_no_search['messages'][-1].content
# print(response_text[:400] + "..." if len(response_text) > 400 else response_text)
# print("-"*70)
# print(f"⏱️  Time: {elapsed_no_search:.2f}s")
# print(f"\n⚠️  Note: Generic answer, likely outdated or vague (LLM training data cutoff)")

# Now create agent WITH search tool
print("\n" + "=" * 70)
print("TEST 2: Agent WITH Search")
print("=" * 70)

# Create new agent with Valyu search tool (uses Holistic AI Bedrock by default)
llm_with_tools = get_chat_model("claude-3-5-sonnet")
agent_with_search = create_react_agent(
    llm_with_tools,
    tools=[search_tool],  # Add the search tool!
)

print(f"\n❓ Same Question: {recent_question}\n")

start_time = time.time()
result_with_search = agent_with_search.invoke({"messages": [HumanMessage(content=recent_question)]})
elapsed_with_search = time.time() - start_time

print("💬 Response (WITH search):")
print("-" * 70)
response_text = result_with_search["messages"][-1].content
print(response_text[:600] + "..." if len(response_text) > 600 else response_text)
print("-" * 70)
print(f"⏱️  Time: {elapsed_with_search:.2f}s")
print("\n✅ Note: Real-time data, specific sources, up-to-date information!")


TEST 2: Agent WITH Search

❓ Same Question: What are the latest breakthroughs in quantum computing in 2025?



/var/folders/gt/r39m3yss08l05rhcqrtkyys40000gn/T/ipykernel_1581/3187584785.py:30: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_with_search = create_react_agent(


💬 Response (WITH search):
----------------------------------------------------------------------
Based on the search results, I can provide a summary of the key quantum computing breakthroughs and developments in 2024-2025:

Key Breakthroughs:

1. Google's Willow Chip (Late 2024):
- Achieved significant improvement in error correction and noise reduction
- Demonstrated ability to perform computations that would take classical supercomputers extremely long periods to solve
- Represents progress toward threshold scalability, though still not ready for practical applications

2. Microsoft's Developments:
- Unveiled the world's first quantum processor powered by topological qubits
- Integrate...
----------------------------------------------------------------------
⏱️  Time: 55.61s

✅ Note: Real-time data, specific sources, up-to-date information!


In [8]:
print("📊 Message Trace (WITH search):")
print("=" * 70)

for i, msg in enumerate(result_with_search["messages"]):
    msg_type = type(msg).__name__
    print(f"\n{i + 1}. {msg_type}")
    print("-" * 70)

    if msg_type == "HumanMessage":
        print(f"   User: {msg.content[:100]}...")

    elif msg_type == "AIMessage":
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"   AI decided to call tool: {msg.tool_calls[0]['name']}")
            print(f"   Args: {msg.tool_calls[0]['args']}")
        else:
            content_preview = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
            print(f"   AI response: {content_preview}")

    elif msg_type == "ToolMessage":
        content_preview = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
        print(f"   Tool returned: {content_preview}")

print("\n" + "=" * 70)
print(f"Total messages: {len(result_with_search['messages'])}")
print("\nThis shows the complete ReAct loop: User → AI (tool call) → Tool → AI (answer)")

📊 Message Trace (WITH search):

1. HumanMessage
----------------------------------------------------------------------
   User: What are the latest breakthroughs in quantum computing in 2025?...

2. AIMessage
----------------------------------------------------------------------
   AI decided to call tool: valyu_deep_search
   Args: {'query': 'major breakthroughs developments quantum computing 2025', 'start_date': '2025-01-01', 'end_date': '2025-12-31', 'max_num_results': 10}

3. ToolMessage
----------------------------------------------------------------------
   Tool returned: {
  "success": false,
  "error": "HTTP Error: 504",
  "tx_id": "error-504",
  "query": "major breakt...

4. AIMessage
----------------------------------------------------------------------
   AI decided to call tool: valyu_deep_search
   Args: {'query': 'latest breakthroughs developments quantum computing 2024', 'max_num_results': 10}

5. ToolMessage
-------------------------------------------------------------

In [29]:
from pydantic import BaseModel, Field
from typing import List


# Define output schema
class Entity(BaseModel):
    """Person, group, or institution mentioned in the article. Contains information and evidence about bias presented."""

    name: str = Field(description="Name of person, group, institution, or organization mentioned")
    tone: str = Field(
        description="Sentiment within the article towards the entity mentioned: MUST be one of the following: 'very positive', 'positive', 'neutral', 'negative', 'very negative'"
    )
    evidence_sentences: List[str] = Field(
        description="Phrases or sentences pulled directly from the provided article that demonstrate the sentiment of the article toward this entity"
    )
    loaded_phrases: List[str] = Field(
        description="Emotionally charged or manipulative wording or phrases used in the article pertaining to the entity"
    )


class EntityAnalysis(BaseModel):
    """Analysis of the entities discussed in the article and how the sentiment presented toward them."""

    entities: List[Entity] = Field(description="All of the people, groups, or institutions mentioned in the article.")


print("Schema defined!")
print(f"   Fields: {list(EntityAnalysis.model_fields.keys())}")
print(f"   Entity Fields: {list(Entity.model_fields.keys())}")
print("   Validation: Built-in with Pydantic")

Schema defined!
   Fields: ['entities']
   Entity Fields: ['name', 'tone', 'evidence_sentences', 'loaded_phrases']
   Validation: Built-in with Pydantic


In [30]:
# Create base LLM
# Use get_chat_model() - uses Holistic AI Bedrock by default (recommended)
llm = get_chat_model("claude-3-5-sonnet")  # Uses Holistic AI Bedrock (recommended)

# Create LLM with structured output
llm_structured = llm.with_structured_output(EntityAnalysis)
print("✅ LLM with structured output created!")
print("   Model: claude-3-5-sonnet (via Bedrock)")
print("   Output format: TechAnalysis (JSON)")
print("   Validation: Automatic with Pydantic")

✅ LLM with structured output created!
   Model: claude-3-5-sonnet (via Bedrock)
   Output format: TechAnalysis (JSON)
   Validation: Automatic with Pydantic


In [32]:
# Test structured output
question = 'You are an entity-level bias detector.\n\nIdentify all people, groups, or institutions discussed in the article.\nFor each entity, determine the tone:\n\'very_positive\', \'positive\', \'neutral\', \'negative\', \'very_negative\'\n\nExtract:\n- 1–3 evidence sentences that reflect this tone\n- any "loaded phrases" (emotionally charged or manipulative wording)\n\nArticle:\n"More than 300 people gathered in Bournemouth earlier as a planned immigration protest took place.\nThe march, organised by Bournemouth Patriots, with a "stop the boats" message, followed a pre-agreed route along Meyrick Road to a designated location near the Lansdowne.\n\nCounter protesters and supporters of Stand up to Racism were also present, with many chanting "refugees are welcome here".\n\nDorset Police said it respected people\'s right to peaceful protest, but were "imposing conditions for safety" and to "minimise disruption".\n\nBBC reporter Steve Humphrey said about 300 people gathered at the height of the protest, with equal numbers from both sides.\n\nHe said some were chanting, while waving Union Jack and England flags.\n\nIt was the latest in a series of protests in the town, with police warning demonstrators would be committing an offence if they failed to follow official instructions.\n\nThe Bournemouth Patriots turned down an invitation to be interviewed by the BBC and said the corporation was not welcome at the event."'

# question = "You are an entity-level bias detector.\n\nIdentify all people, groups, or institutions discussed in the article.\nFor each entity, determine the tone:\n'very_positive', 'positive', 'neutral', 'negative', 'very_negative'\n\nExtract:\n- 1–3 evidence sentences that reflect this tone\n- any \"loaded phrases\" (emotionally charged or manipulative wording)\n\nArticle:\n\"The Strategic Uncoupling of Zohran Mamdani and Brad Lander\n\nMr. Lander, the New York City comptroller, campaigned with Mr. Mamdani and once hoped to join him in City Hall. Now he is eyeing a congressional seat. Zohran Mamdani was in the final, chaotic sprint to Election Day in the New York City mayor’s race, when he stepped off the campaign trail for an important meeting.\n\nSitting with him was Brad Lander, the city comptroller who had become one of his closest allies. Their \“progressive bromance\” during the Democratic primary in June had helped cinch Mr. Mamdani\’s victory and left Mr. Lander gunning for City Hall, too, hopefully as his top deputy.\n\nBut when they met on a chaotic Sunday in late October, amid church services and canvass launches, Mr. Mamdani, a state assemblyman, wanted to talk about a different idea that Mr. Lander had floated. He told the comptroller that he would like to continue their partnership — by supporting him in a primary challenge against Representative Daniel Goldman, a vulnerable and more moderate Democrat, according to three people familiar with the exchange.\n\nAs for the high-level administration post Mr. Lander, 56, had also coveted, Mr. Mamdani told him that he planned to go in a different direction.The sharp change-up has captivated New York’s political chattering class since it spilled into public in recent days, setting the stage for a potentially explosive House primary in the heart of New York City.\n\nMr. Mamdani\’s quiet maneuvering — much of which has not been previously reported — has also offered an early window into the unsentimental calculations guiding the incoming mayor as he builds his administration and flexes his political muscle.\n\nAsked to comment for this story, Mr. Mamdani’s spokeswoman, Dora Pekec, said simply that Mr. Lander “continues to be a trusted ally and partner to the mayor-elect.” But other supporters who have spoken to him added that Mr. Mamdani, a sharp critic of Israel, was eager to unseat Mr. Goldman, whose views on the war in Gaza and other issues are well to his right.\n\nPeople who have spoken with Mr. Lander recently said he had played down the sting of being passed over for a top city post.\n\n\“Brad\’s moral clarity, his willingness to use his voice to defend our democracy and put his body on the line to protect our neighbors are all vitally important and unfortunately all too rare,” said his top adviser, Alison Hirsh. She said he would bring these qualities to Washington “if he chooses to run.”\n\nThe relationship between Mr. Lander and Mr. Mamdani has been the subject of unusual interest since the two men rolled out a novel cross-endorsement in June as competing candidates for mayor. Politicians often talk about forming alliances to defeat a common enemy, in this case former Gov. Andrew M. Cuomo, but rarely follow through.Mr. Lander, a well-known progressive with 15 years in public office, vouched for Mr. Mamdani, a 34-year-old Muslim and democratic socialist, with fellow Jewish Democrats. And after federal agents arrested Mr. Lander as he escorted migrants in an immigration courthouse, he used some of the attention to boost his formal rival.\n\nOn the night of his primary victory, Mr. Mamdani put his arm around Mr. Lander and said they had modeled \“the politics of the future, one of partnership and of sincerity.\”\n\nMany progressive New Yorkers supporting Mr. Mamdani — but uncertain about his youth with just five years in the State Assembly under his belt — felt comforted by the prospect of Mr. Lander joining City Hall as a partner overseeing the city’s day-to-day operations.\""

print(f"Question: {question}\n")
print("=" * 70)

start_time = time.time()
result = llm_structured.invoke(question)
elapsed = time.time() - start_time

print("Structured Output (Pydantic Object):")
print("=" * 70)
for i, entity in enumerate(result.entities, 1):
    print(f"  {i}. {entity}")

print("=" * 70)
print(f"Time: {elapsed:.2f}s")

Question: You are an entity-level bias detector.

Identify all people, groups, or institutions discussed in the article.
For each entity, determine the tone:
'very_positive', 'positive', 'neutral', 'negative', 'very_negative'

Extract:
- 1–3 evidence sentences that reflect this tone
- any "loaded phrases" (emotionally charged or manipulative wording)

Article:
"More than 300 people gathered in Bournemouth earlier as a planned immigration protest took place.
The march, organised by Bournemouth Patriots, with a "stop the boats" message, followed a pre-agreed route along Meyrick Road to a designated location near the Lansdowne.

Counter protesters and supporters of Stand up to Racism were also present, with many chanting "refugees are welcome here".

Dorset Police said it respected people's right to peaceful protest, but were "imposing conditions for safety" and to "minimise disruption".

BBC reporter Steve Humphrey said about 300 people gathered at the height of the protest, with equal n